<a href="https://colab.research.google.com/github/dibya001/superres/blob/master/superres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pillow>=5.0.0


In [0]:
!pip install tensorflow>=1.13.1

In [0]:
!pip install numpy>=1.16.0

In [0]:
!pip install wandb>=0.8.0

In [0]:
!wandb login

You can find your API keys in your browser here: https://app.wandb.ai/authorize
Paste an API key from your profile: 75aaa66c376d67516bcd56673b0eaa4d27cce6f8
Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [0]:
import random
import glob
import subprocess
import os
from PIL import Image
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import Callback
import wandb
from wandb.keras import WandbCallback

run = wandb.init(project='superres')
config = run.config


In [0]:
config.num_epochs = 20
config.batch_size = 30
config.input_height = 32
config.input_width = 32
config.output_height = 256
config.output_width = 256

val_dir = 'data/test'
train_dir = 'data/train'

# automatically get the data if it doesn't exist
if not os.path.exists("data"):
    print("Downloading flower dataset...")
    subprocess.check_output(
        "mkdir data && curl https://storage.googleapis.com/wandb/flower-enhance.tar.gz | tar xz -C data", shell=True)

config.steps_per_epoch = len(
    glob.glob(train_dir + "/*-in.jpg")) // config.batch_size
config.val_steps_per_epoch = len(
    glob.glob(val_dir + "/*-in.jpg")) // config.batch_size


def image_generator(batch_size, img_dir):
    """A generator that returns small images and large images.  DO NOT ALTER the validation set"""
    input_filenames = glob.glob(img_dir + "/*-in.jpg")
    counter = 0
    random.shuffle(input_filenames)
    while True:
        small_images = np.zeros(
            (batch_size, config.input_width, config.input_height, 3))
        large_images = np.zeros(
            (batch_size, config.output_width, config.output_height, 3))
        if counter+batch_size >= len(input_filenames):
            counter = 0
        for i in range(batch_size):
            img = input_filenames[counter + i]
            small_images[i] = np.array(Image.open(img)) / 255.0
            large_images[i] = np.array(
                Image.open(img.replace("-in.jpg", "-out.jpg"))) / 255.0
        yield (small_images, large_images)
        counter += batch_size


In [0]:
def perceptual_distance(y_true, y_pred):
    """Calculate perceptual distance, DO NOT ALTER"""
    y_true *= 255
    y_pred *= 255
    rmean = (y_true[:, :, :, 0] + y_pred[:, :, :, 0]) / 2
    r = y_true[:, :, :, 0] - y_pred[:, :, :, 0]
    g = y_true[:, :, :, 1] - y_pred[:, :, :, 1]
    b = y_true[:, :, :, 2] - y_pred[:, :, :, 2]

    return K.mean(K.sqrt((((512+rmean)*r*r)/256) + 4*g*g + (((767-rmean)*b*b)/256)))


In [0]:
val_generator = image_generator(config.batch_size, val_dir)
in_sample_images, out_sample_images = next(val_generator)


In [0]:
class ImageLogger(Callback):
    def on_epoch_end(self, epoch, logs):
        preds = self.model.predict(in_sample_images)
        in_resized = []
        for arr in in_sample_images:
            # Simple upsampling
            in_resized.append(arr.repeat(8, axis=0).repeat(8, axis=1))
        wandb.log({
            "examples": [wandb.Image(np.concatenate([in_resized[i] * 255, o * 255, out_sample_images[i] * 255], axis=1)) for i, o in enumerate(preds)]
        }, commit=False)


In [0]:
model = Sequential()
model.add(layers.Conv2D(128, (5, 5), activation='relu', padding='same',
                        input_shape=(config.input_width, config.input_height, 3)))
model.add(layers.Conv2D(64, (5, 5), activation='relu', padding='same'))
model.add(layers.Conv2DTranspose(64, (5,5), strides = (2,2), padding = 'same', use_bias = False))
model.add(layers.Conv2D(32, (5, 5), activation='relu', padding='same'))
model.add(layers.Conv2DTranspose(32, (5,5), strides = (2,2), padding = 'same', use_bias = False))
model.add(layers.Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(layers.Conv2DTranspose(16, (5,5), strides = (2,2), padding = 'same', use_bias = False))
model.add(layers.Conv2D(3, (3, 3), activation='relu', padding='same'))


W0626 08:21:11.951749 139646910982016 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
# DONT ALTER metrics=[perceptual_distance]
model.compile(optimizer='adam', loss='mse',
              metrics=[perceptual_distance])

In [13]:
model.fit_generator(image_generator(config.batch_size, train_dir),
                    steps_per_epoch=config.steps_per_epoch,
                    epochs=config.num_epochs, callbacks=[
                        ImageLogger(), WandbCallback()],
                    validation_steps=config.val_steps_per_epoch,
                    validation_data=val_generator)


Epoch 1/20
166/166 [==============================] - 47s 281ms/step - loss: 0.0776 - perceptual_distance: 172.4713 - val_loss: 0.0661 - val_perceptual_distance: 155.4858
Epoch 2/20
166/166 [==============================] - 43s 262ms/step - loss: 0.0303 - perceptual_distance: 96.0422 - val_loss: 0.0156 - val_perceptual_distance: 68.0878
Epoch 3/20
166/166 [==============================] - 44s 263ms/step - loss: 0.0152 - perceptual_distance: 66.3686 - val_loss: 0.0151 - val_perceptual_distance: 66.0877
Epoch 4/20
166/166 [==============================] - 44s 264ms/step - loss: 0.0153 - perceptual_distance: 67.2388 - val_loss: 0.0150 - val_perceptual_distance: 65.9742
Epoch 5/20
166/166 [==============================] - 44s 265ms/step - loss: 0.0146 - perceptual_distance: 63.8974 - val_loss: 0.0147 - val_perceptual_distance: 64.2451
Epoch 6/20
166/166 [==============================] - 44s 266ms/step - loss: 0.0146 - perceptual_distance: 64.1092 - val_loss: 0.0147 - val_perceptual_di

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       9728      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        204864    
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 64, 64, 64)        102400    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        51232     
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 128, 128, 32)      25600     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 16)      4624      
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 256, 256, 16)      6